In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import scipy.io as sio
import hdf5storage

Extracts 
1. baseMemoir manually classified labels and corresponding training features, and
2. the baseMemoir features for each site (corresponding to the dynamic barcodes), the intBC calls, and the baseMemoir discriminative classifier probabilities.

In [3]:
fname = "/Users/gc3045/git/laml2-experiments/real_data/baseMemoir/BaseMEM_Magic/Magic/Figures3_5_and_SupFigures2_4-6/all_position_summary_workspace.mat"
fname_svm = "/Users/gc3045/git/laml2-experiments/real_data/baseMemoir/BaseMEM_Magic/Magic/Figures3_5_and_SupFigures2_4-6/lineage_analysis/baseMEM-01_all_sample_final_BCs_SBCfilt_MIN_SVM.txt"

In [4]:
data = hdf5storage.loadmat(fname, simplify_cells=True)

In [5]:
print(list(data.keys()))

['#subsystem#', 'CVMdl', 'Mdl_all_together', 'Mdl_array', 'PCA_mRNA', 'Posterior_array', 'Posterior_array_all_together', 'all_annotated_subsets', 'all_manual_classifications', 'all_positions_array', 'annotated_subset_array', 'ans', 'b', 'barcode_dist_freqs', 'barcode_dists', 'bc_whitelist', 'cell_assignments', 'cell_mask_array', 'cell_pos_dot_labels', 'cell_pos_dot_labels_nofilt', 'cell_pos_labels', 'cell_pos_sBC_labels', 'cell_pos_sBC_labels_nofilt', 'cells_list', 'class_fractions_all', 'class_fractions_dynamic', 'class_fractions_static', 'class_labs', 'class_labs_all_together', 'colors', 'confmat', 'ctr', 'curr_cell', 'curr_cell_mask', 'curr_dots_masked', 'curr_mask', 'd', 'dot_count', 'dot_pos_labels', 'dynamic_bc_calls', 'explained', 'final_bcs', 'final_bcs_dists', 'final_bcs_filt', 'final_bcs_fives', 'final_bcs_nofilt', 'final_consensus_array', 'final_dot_matrix', 'final_dot_matrix_probs', 'first_cell_measures', 'gene', 'gene_idx', 'hBar', 'i', 'idx', 'int_bc_freqs', 'j', 'joint_b

In [7]:
label_to_str = {1: "GG", 2: "AA", 3: "GA", 4: "AG"}
new_str_to_label = {"AA": 0, "GG": 1, "GA": 2, "AG": 3}

#### Save the training data

In [8]:
manual_training_labels = data['all_manual_classifications'] # training labels (9598, 1)
manual_training_data = pd.DataFrame(data['all_annotated_subsets'][:, 4:20]) # training feature vectors (9598, 24) subset to (9598, 16)

In [9]:
manual_training_data.columns = [f"feature_{x}" for x in range(len(manual_training_data.columns))]

In [10]:
manual_training_data['baseMemoir_cat_label'] = [int(x) for x in manual_training_labels]

/var/folders/jx/nl4vbs_j5qdbrq_3w4lqgcwc0000gn/T/ipykernel_97521/3391046385.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  manual_training_data['baseMemoir_cat_label'] = [int(x) for x in manual_training_labels]


In [11]:
manual_training_data["str_label"] = manual_training_data["baseMemoir_cat_label"].map(label_to_str)
manual_training_data = manual_training_data.drop(columns=["baseMemoir_cat_label"])
manual_training_data["label"] = manual_training_data["str_label"].map(new_str_to_label)

In [12]:
manual_training_data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,str_label,label
0,8.390723,10.589963,9.195633,8.135347,4.499810,6.746412,5.940171,4.394449,5.351858,7.375882,6.700731,5.010635,8.537349,12.402022,10.932860,7.717133,AA,0
1,6.795706,10.183049,6.492240,7.297768,0.693147,6.499787,0.000000,1.791759,4.007333,7.265430,3.828641,4.382027,6.528028,12.014996,6.647458,7.296454,AA,0
2,3.091042,8.136518,3.912023,6.629363,0.000000,0.000000,0.000000,4.615121,2.639057,6.579251,3.258097,4.890349,3.504592,11.328031,4.777772,7.156956,AA,0
3,8.743053,5.313206,8.057061,4.700480,5.840642,0.000000,5.533389,0.000000,6.293419,3.218876,6.052089,0.000000,9.883953,5.112581,9.904880,4.647009,GG,1
4,4.043051,9.337149,6.813445,4.615121,0.000000,6.142037,3.713572,0.000000,2.639057,6.923629,4.605170,2.995732,3.439405,11.417102,7.275389,4.884794,AA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9593,4.406719,4.510860,10.631640,4.934474,0.000000,1.791759,7.487174,0.000000,3.637586,3.555348,8.492080,4.158883,5.533883,5.242880,14.837857,6.622071,GA,2
9594,9.523690,5.262690,7.030857,6.746412,6.124683,0.000000,0.000000,0.000000,6.594413,2.564949,4.007333,4.043051,10.480612,3.616507,6.733781,6.438467,GG,1
9595,6.803505,8.604288,9.371523,13.975476,1.386294,3.850148,5.023881,9.550449,3.637586,5.241747,5.808142,10.567103,5.544121,8.609815,9.642996,18.871455,AG,3
9596,5.407172,5.676754,6.941190,10.777810,0.000000,1.945910,6.249975,7.540622,3.295837,3.583519,6.489205,8.098339,5.165890,5.609130,10.583587,13.879680,AG,3


In [13]:
manual_training_data.to_csv("/Users/gc3045/git/laml2-experiments/real_data/baseMemoir/inputs/baseMemoir_training_data.csv", index=False)

#### Save the test data

In [14]:
columns = (
    [f"site{t}_p_state1" for t in range(1, 7)] +
    [f"site{t}_p_state2" for t in range(1, 7)] +
    [f"site{t}_p_state3" for t in range(1, 7)] +
    [f"site{t}_p_state4" for t in range(1, 7)] +
    ["final_bcs","final_bcs_dists","final_bcs_fives","cell_label","dot_index","position"] +
    [f"site{t}_det_call" for t in range(1, 7)] +
    ["duplicate_flag"] +
    [f"site{t}_p_max" for t in range(1, 7)]
)

In [15]:
baseMemoir_geno_df = pd.read_csv(fname_svm, header=None, sep='\t')
baseMemoir_geno_df.columns = columns

In [16]:
baseMemoir_geno_df

,site1_p_state1,site2_p_state1,site3_p_state1,site4_p_state1,site5_p_state1,site6_p_state1,site1_p_state2,site2_p_state2,site3_p_state2,site4_p_state2,...,site4_det_call,site5_det_call,site6_det_call,duplicate_flag,site1_p_max,site2_p_max,site3_p_max,site4_p_max,site5_p_max,site6_p_max
0,0.909489,0.350080,0.010919,0.023386,0.567286,0.013363,0.001505,0.000201,0.940735,0.942045,...,2,1,2,0,0.909489,0.644814,0.940735,0.942045,0.567286,0.967499
1,0.058301,0.163961,0.025379,0.005846,0.792200,0.031266,0.939864,0.747836,0.974346,0.989070,...,2,1,2,0,0.939864,0.747836,0.974346,0.989070,0.792200,0.963389
2,0.500774,0.619704,0.115522,0.036347,0.600173,0.392341,0.425164,0.371873,0.824186,0.301438,...,2,2,2,1,0.500774,0.619704,0.824186,0.649125,0.600173,0.603349
3,0.005898,0.060270,0.009903,0.002153,0.981871,0.334341,0.017757,0.087983,0.930804,0.115522,...,4,1,4,0,0.836904,0.526965,0.930804,0.871806,0.981871,0.421716
4,0.959972,0.053203,0.004680,0.000433,0.002294,0.004599,0.011470,0.946145,0.989062,0.001238,...,3,2,2,0,0.959972,0.946145,0.989062,0.998327,0.997493,0.993820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15073,0.500774,0.619704,0.115522,0.036347,0.600173,0.392341,0.425164,0.371873,0.824186,0.301438,...,2,1,1,1,0.500774,0.619704,0.824186,0.649125,0.600173,0.603349
15074,0.158512,0.001431,0.016342,0.006806,0.062592,0.337484,0.806082,0.981631,0.933280,0.950460,...,2,2,2,0,0.806082,0.981631,0.933280,0.950460,0.827910,0.656545
15075,0.340758,0.416640,0.081612,0.024495,0.404222,0.270054,0.586714,0.510968,0.771208,0.401894,...,2,4,2,1,0.586714,0.510968,0.771208,0.552637,0.486021,0.586569
15076,0.005315,0.001796,0.007234,0.022292,0.177742,0.013363,0.994666,0.981387,0.952709,0.963016,...,2,2,2,0,0.994666,0.981387,0.952709,0.963016,0.812826,0.984542


In [17]:
print("Mean baseMemoir classification confidence:", baseMemoir_geno_df[[f"site{t}_p_max" for t in range(1, 7)]].mean(axis=0).mean())

cols = [f"site{t}_p_max" for t in range(1, 7)]
thresh = 0.7
per_site_pct = (baseMemoir_geno_df[cols] <= thresh).mean(axis=0) * 100
overall_pct = per_site_pct.mean()

print(f"%% at each site <= {thresh} confidence:")
print(per_site_pct.round(1).to_string())
print(f"Overall mean across sites: {overall_pct:.1f}%")

cols = ['final_bcs', 'final_bcs_dists', 'final_bcs_fives', 'cell_label', 'dot_index', 'position']
ranges = baseMemoir_geno_df[cols].agg(["min", "max"]).T
print(ranges)

Mean baseMemoir classification confidence: 0.8505443352601604
%% at each site <= 0.7 confidence:
site1_p_max    29.1
site2_p_max    31.8
site3_p_max     6.1
site4_p_max    28.6
site5_p_max    27.9
site6_p_max    26.2
Overall mean across sites: 25.0%
                 min   max
final_bcs          1    66
final_bcs_dists    0     2
final_bcs_fives    0     0
cell_label         1    74
dot_index          1  4545
position           0    10


In [42]:
# reformat into "long" format, so that I can add features for each site
prob_col_order = [f"p_{label_to_str[k]}" for k in [1, 2, 3, 4]]  # p_GG, p_AA, p_GA, p_AG

meta = ["final_bcs","final_bcs_dists","final_bcs_fives",
        "cell_label","dot_index","position","duplicate_flag"]

df = baseMemoir_geno_df.copy()
df["_row"] = np.arange(len(df))

# melt siteX_p_stateY -> (site, state, prob)
prob_cols = [c for c in df.columns if re.match(r"site\d+_p_state\d+$", c)]
probs_long = (
    df.melt(id_vars=meta + ["_row"], value_vars=prob_cols,
            var_name="site_state", value_name="prob")
      .assign(
          site=lambda d: d["site_state"].str.extract(r"site(\d+)", expand=False).astype(int),
          state=lambda d: d["site_state"].str.extract(r"p_state(\d+)", expand=False).astype(int)
      )
      .drop(columns=["site_state"])
)

long_df = (
    probs_long
      .pivot_table(index=meta + ["_row", "site"],
                   columns="state", values="prob", aggfunc="first")
      .reset_index()
      # map 1..4 -> p_GG, p_AA, p_GA, p_AG
      .rename(columns={k: f"p_{v}" for k, v in label_to_str.items()})
      # ensure canonical column order for the four probs
      .reindex(columns=meta + ["_row","site"] + prob_col_order, fill_value=np.nan)
      .sort_values(["_row", "site"], ignore_index=True)
)
long_df = long_df.reset_index(drop=True)

In [43]:
prob_cols = ['p_GG', 'p_AA', 'p_GA', 'p_AG']

long_df['baseMemoir_genotype'] = (long_df[prob_cols].idxmax(axis=1).str.replace('p_', '', regex=False))
long_df['baseMemoir_pmax'] = long_df[prob_cols].max(axis=1)

long_df

state,final_bcs,final_bcs_dists,final_bcs_fives,cell_label,dot_index,position,duplicate_flag,_row,site,p_GG,p_AA,p_GA,p_AG,baseMemoir_genotype,baseMemoir_pmax
0,1,0,0,1,57,1,0,0,1,0.909489,0.001505,0.087431,0.001574,GG,0.909489
1,1,0,0,1,57,1,0,0,2,0.350080,0.000201,0.004905,0.644814,AG,0.644814
2,1,0,0,1,57,1,0,0,3,0.010919,0.940735,0.046383,0.001963,AA,0.940735
3,1,0,0,1,57,1,0,0,4,0.023386,0.942045,0.032878,0.001690,AA,0.942045
4,1,0,0,1,57,1,0,0,5,0.567286,0.004091,0.311299,0.117325,GG,0.567286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90463,9,1,0,9,649,9,0,15077,2,0.012860,0.979866,0.000210,0.007064,AA,0.979866
90464,9,1,0,9,649,9,0,15077,3,0.010284,0.987961,0.000639,0.001116,AA,0.987961
90465,9,1,0,9,649,9,0,15077,4,0.001472,0.966041,0.032212,0.000274,AA,0.966041
90466,9,1,0,9,649,9,0,15077,5,0.664211,0.022644,0.000001,0.313143,GG,0.664211


Now we need to extract the features and map them to the calls above based on the metadata. The features are in the same order in both `tidy_dots_filt_array_all` and `all_annotated_subsets(:,5:20)`, see

```
Mdl_all_together = fitcecoc(all_annotated_subsets(:,5:20), ...
        all_manual_classifications,...
        'Learners', t, 'FitPosterior', true, 'Verbose',2);
```
and
```
[class_labs_all_together, ~, ~, probs_all_together] = ...
    predict(Mdl_all_together, tidy_dots_filt_array_all(:,5:20));
```

In [21]:
cols = (
    ['site_id', 'dot_idx', 'cell_id', 'position'] +
    [f'feature_{x}' for x in range(16)]
    # [f'state{s}_r{r}' for r in range(1, 5) for s in range(1, 5)]
)
test_features = pd.DataFrame(data['tidy_dots_filt_array_all'])
test_features = test_features.iloc[:, :20].copy()   # <- use .iloc
test_features.columns = cols

# only take the observations for dynamic sites
df_dyn = test_features[test_features['site_id'].between(1, 6)].reset_index(drop=True)
df_dyn

,site_id,dot_idx,cell_id,position,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,1.0,28.0,3.0,0.0,2.639057,7.575585,5.973810,5.638355,0.000000,4.744932,3.218876,2.772589,0.000000,5.579730,4.158883,3.912023,2.447767,9.393669,6.541415,5.870614
1,1.0,30.0,3.0,0.0,8.585039,5.971262,5.723585,6.107023,5.823046,0.000000,0.000000,0.000000,6.175867,3.713572,3.401197,3.761200,9.420757,5.750204,5.752739,5.814338
2,1.0,31.0,3.0,0.0,6.285998,8.701845,6.206576,6.598509,2.772589,5.883322,6.206576,1.386294,3.610918,6.617403,6.206576,4.553877,5.456836,11.007664,0.000000,7.106810
3,1.0,33.0,3.0,0.0,6.621406,10.759731,10.845427,8.321665,0.693147,6.878326,6.806829,5.860786,3.496508,7.215240,7.173192,6.156979,5.260872,11.538437,11.288487,8.621989
4,1.0,34.0,3.0,0.0,7.726213,4.521789,4.234107,4.477337,4.867534,0.693147,0.000000,0.000000,5.525453,2.890372,2.833213,3.555348,8.655293,4.485213,4.296832,5.182883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147319,6.0,3043.0,36.0,10.0,5.758902,9.520689,5.087596,5.497168,0.000000,6.016157,0.000000,0.000000,3.044522,6.902743,3.135494,2.944439,4.543898,11.554306,4.271327,4.888237
147320,6.0,3049.0,36.0,10.0,12.310707,2.995732,7.037906,6.654153,8.108623,0.000000,1.386294,0.000000,8.805225,0.000000,4.276666,3.637586,15.043771,0.999105,7.000213,6.076146
147321,6.0,3053.0,39.0,10.0,4.094345,7.802618,5.976351,5.707110,0.000000,5.846439,1.098612,2.484907,2.484907,6.150603,3.931826,3.526361,3.085013,8.988315,6.559900,5.345657
147322,6.0,3062.0,39.0,10.0,5.476464,8.903408,6.249975,4.828314,0.000000,5.802118,3.091042,0.000000,2.890372,6.505784,4.290459,2.197225,4.403772,10.843625,6.733585,3.922675


In [22]:
cols = ['site_id', 'cell_id', 'dot_idx', 'position']
ranges = df_dyn[cols].agg(["min", "max"]).T
print(ranges)

          min     max
site_id   1.0     6.0
cell_id   1.0    74.0
dot_idx   1.0  4546.0
position  0.0    10.0


In [23]:
df_dyn = df_dyn.copy()
long_df = long_df.copy()

keys = ['site','cell_label','position','dot_index']

for c in ['site_id','cell_id','dot_idx','position']:
    df_dyn[c] = pd.to_numeric(df_dyn[c], errors='coerce').astype('Int64')

for c in ['site','cell_label','dot_index','position']:
    long_df[c] = pd.to_numeric(long_df[c], errors='coerce').astype('Int64')

feat_cols = [c for c in df_dyn.columns if c.startswith('feature_')]
rhs = (
    df_dyn
    .rename(columns={'site_id':'site', 'cell_id':'cell_label', 'dot_idx':'dot_index'})
    [['site','cell_label','dot_index','position'] + feat_cols]
)

# -- 3) Merge (left join keeps all rows in long_df)
merged = long_df.merge(rhs, on=['site','cell_label','dot_index','position'],
                       how='left', validate='1:1')

# (Optional) quick sanity check for unmatched rows
unmatched = merged[feat_cols].isna().all(axis=1).sum()
print(f'Unmatched rows (no feature found): {unmatched} / {len(merged)}')

left_keys  = long_df[keys].drop_duplicates()
right_keys = rhs[keys].drop_duplicates()

missing = (
    left_keys.merge(right_keys, on=keys, how='left', indicator=True)
             .query('_merge == "left_only"')
             .drop(columns=['_merge'])
)

print(f"[dot_index check] Missing feature rows (no exact key match): {len(missing)}")
if len(missing):
    print(missing.head())  # show a few to debug

ambig_counts = (
    rhs.groupby(['site','cell_label','position'])['dot_index']
       .nunique()
       .rename('n_dot_index')
       .reset_index()
)

ambiguous_keys = ambig_counts[ambig_counts['n_dot_index'] > 1]
print(f"[dot_index check] Ambiguous keys in df_dyn (multiple dot_index per site/cell/pos): {len(ambiguous_keys)}")
if len(ambiguous_keys):
    print(ambiguous_keys.head())

Unmatched rows (no feature found): 0 / 90468
[dot_index check] Missing feature rows (no exact key match): 0
[dot_index check] Ambiguous keys in df_dyn (multiple dot_index per site/cell/pos): 2322
   site  cell_label  position  n_dot_index
0     1           1         1           64
1     1           1         2           94
2     1           1         3           94
3     1           1         4           75
4     1           1         5           70


In [24]:
merged.columns

Index(['final_bcs', 'final_bcs_dists', 'final_bcs_fives', 'cell_label',
       'dot_index', 'position', 'duplicate_flag', '_row', 'site', 'p_GG',
       'p_AA', 'p_GA', 'p_AG', 'baseMemoir_genotype', 'baseMemoir_pmax',
       'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15'],
      dtype='object')

In [25]:
merged

,final_bcs,final_bcs_dists,final_bcs_fives,cell_label,dot_index,position,duplicate_flag,_row,site,p_GG,...,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,1,0,0,1,57,1,0,0,1,0.909489,...,6.432940,3.465736,7.872455,5.998937,7.137278,4.343805,13.414877,10.442790,12.074642,6.916006
1,1,0,0,1,57,1,0,0,2,0.350080,...,4.488636,7.465083,7.734121,4.859812,5.459586,8.599879,13.139092,7.979672,9.143101,14.925368
2,1,0,0,1,57,1,0,0,3,0.010919,...,6.175867,4.077537,3.951244,7.779467,6.280396,4.644391,6.160170,13.279019,9.722703,7.240190
3,1,0,0,1,57,1,0,0,4,0.023386,...,5.897154,0.000000,3.713572,7.662468,6.453625,4.158883,5.673921,12.907225,10.776704,8.745427
4,1,0,0,1,57,1,0,0,5,0.567286,...,6.490724,6.373320,7.384610,5.537334,7.593374,7.568379,12.488600,9.625884,13.112572,12.945165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90463,9,1,0,9,649,9,0,15077,2,0.012860,...,2.772589,4.007333,4.110874,7.425954,4.941642,5.752573,6.309114,12.192291,8.228100,9.489516
90464,9,1,0,9,649,9,0,15077,3,0.010284,...,3.496508,2.995732,4.356709,7.643483,5.241747,4.927254,6.896179,13.099163,8.942562,8.210779
90465,9,1,0,9,649,9,0,15077,4,0.001472,...,4.934474,0.000000,3.091042,7.553287,6.059123,3.295837,4.280324,12.767015,9.989126,6.528503
90466,9,1,0,9,649,9,0,15077,5,0.664211,...,0.000000,6.797940,7.218177,5.730100,3.178054,7.552237,12.141150,9.338231,5.023106,12.887858


In [26]:
df_dyn = df_dyn.copy()
long_df = long_df.copy()

for c in ['site_id','cell_id','dot_idx','position']:
    if c in df_dyn.columns:
        df_dyn[c] = pd.to_numeric(df_dyn[c], errors='coerce').astype('Int64')

for c in ['site','cell_label','dot_index','position','duplicate_flag']:
    if c in long_df.columns:
        long_df[c] = pd.to_numeric(long_df[c], errors='coerce').astype('Int64')

# ---- 1) Right-hand side with features and a consistent key ----
feat_cols = [c for c in df_dyn.columns if c.startswith('feature_')]
rhs = (
    df_dyn.rename(columns={'site_id':'site', 'cell_id':'cell_label', 'dot_idx':'dot_index'})[
        ['site','cell_label','dot_index','position'] + feat_cols
    ]
)

# ---- 2) Exact merge (includes dot_index) just like before ----
merged = long_df.merge(rhs, on=['site','cell_label','dot_index','position'],
                       how='left', validate='1:1')

unmatched = merged[feat_cols].isna().all(axis=1).sum()
print(f'Unmatched rows (no feature found on exact key): {unmatched} / {len(merged)}')

# ---- 3) Candidate search ignoring dot_index (detect extra feature vectors) ----
# Use a stable row id from the index so we never depend on a missing column.
long_df_idx = long_df.reset_index(names='orig_idx')

tmp = long_df_idx.merge(
    rhs, on=['site','cell_label','position'], how='left', suffixes=('', '_cand')
)

# keep only rows where we actually received any feature values from df_dyn
has_any_feat = ~tmp[feat_cols].isna().all(axis=1)
tmp2 = tmp[has_any_feat].copy()

# pack each candidate's 16-dim feature vector
tmp2['feat_vec'] = tmp2[feat_cols].to_numpy().tolist()

agg = (
    tmp2.groupby('orig_idx')
        .agg(n_candidates=('dot_index', 'nunique'),
             candidate_dot_index_list=('dot_index', lambda x: sorted(pd.unique(x))),
             features_list=('feat_vec', list))
        .reset_index()
)

long_df_plus = long_df_idx.merge(agg, on='orig_idx', how='left').drop(columns='orig_idx')
long_df_plus['n_candidates'] = long_df_plus['n_candidates'].fillna(0).astype(int)

# dot_index ∈ candidate list?
def _contains(row):
    lst = row['candidate_dot_index_list']
    try:
        return int(row['dot_index']) in list(lst)
    except Exception:
        return False

long_df_plus['dot_is_candidate'] = long_df_plus.apply(_contains, axis=1)

print("Rows with >1 candidate features (ignoring dot_index):",
      (long_df_plus['n_candidates'] > 1).sum())
print("... among duplicate_flag==1:",
      ((long_df_plus['duplicate_flag'] == 1) & (long_df_plus['n_candidates'] > 1)).sum())

# Peek at suspicious rows
suspicious = long_df_plus[
    (long_df_plus['duplicate_flag'] == 1) &
    ((long_df_plus['n_candidates'] > 1) | (~long_df_plus['dot_is_candidate']))
][['site','cell_label','position','dot_index','final_bcs','duplicate_flag',
   'n_candidates','candidate_dot_index_list']]
print(suspicious.head(10))

Unmatched rows (no feature found on exact key): 0 / 90468
Rows with >1 candidate features (ignoring dot_index): 0
... among duplicate_flag==1: 0
Empty DataFrame
Columns: [site, cell_label, position, dot_index, final_bcs, duplicate_flag, n_candidates, candidate_dot_index_list]
Index: []


In [27]:
feat_cols = [c for c in merged.columns if c.startswith('feature_')]

rename_map = {
    'final_bcs': 'cassette_idx',
    'cell_label': 'cell_name',
    'position': 'colony',
    'site': 'site_idx',
    'p_AA': 'bMp_AA',
    'p_GG': 'bMp_GG',
    'p_GA': 'bMp_GA',
    'p_AG': 'bMp_AG',
    'baseMemoir_genotype': 'bM_str_geno',
    'baseMemoir_pmax': 'bM_pmax',
}

cols_to_keep = list(rename_map.keys()) + feat_cols
out = merged[cols_to_keep].rename(columns=rename_map)

meta_order = ['cassette_idx','cell_name','colony','site_idx',
              'bMp_AA','bMp_GG','bMp_GA','bMp_AG','bM_str_geno','bM_pmax']
out = out[meta_order + feat_cols]


In [28]:
out

,cassette_idx,cell_name,colony,site_idx,bMp_AA,bMp_GG,bMp_GA,bMp_AG,bM_str_geno,bM_pmax,...,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,1,1,1,1,0.001505,0.909489,0.087431,0.001574,GG,0.909489,...,6.432940,3.465736,7.872455,5.998937,7.137278,4.343805,13.414877,10.442790,12.074642,6.916006
1,1,1,1,2,0.000201,0.350080,0.004905,0.644814,AG,0.644814,...,4.488636,7.465083,7.734121,4.859812,5.459586,8.599879,13.139092,7.979672,9.143101,14.925368
2,1,1,1,3,0.940735,0.010919,0.046383,0.001963,AA,0.940735,...,6.175867,4.077537,3.951244,7.779467,6.280396,4.644391,6.160170,13.279019,9.722703,7.240190
3,1,1,1,4,0.942045,0.023386,0.032878,0.001690,AA,0.942045,...,5.897154,0.000000,3.713572,7.662468,6.453625,4.158883,5.673921,12.907225,10.776704,8.745427
4,1,1,1,5,0.004091,0.567286,0.311299,0.117325,GG,0.567286,...,6.490724,6.373320,7.384610,5.537334,7.593374,7.568379,12.488600,9.625884,13.112572,12.945165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90463,9,9,9,2,0.979866,0.012860,0.000210,0.007064,AA,0.979866,...,2.772589,4.007333,4.110874,7.425954,4.941642,5.752573,6.309114,12.192291,8.228100,9.489516
90464,9,9,9,3,0.987961,0.010284,0.000639,0.001116,AA,0.987961,...,3.496508,2.995732,4.356709,7.643483,5.241747,4.927254,6.896179,13.099163,8.942562,8.210779
90465,9,9,9,4,0.966041,0.001472,0.032212,0.000274,AA,0.966041,...,4.934474,0.000000,3.091042,7.553287,6.059123,3.295837,4.280324,12.767015,9.989126,6.528503
90466,9,9,9,5,0.022644,0.664211,0.000001,0.313143,GG,0.664211,...,0.000000,6.797940,7.218177,5.730100,3.178054,7.552237,12.141150,9.338231,5.023106,12.887858


In [33]:
out["bM_geno"] = out["bM_str_geno"].map(new_str_to_label)

In [34]:
out.to_csv("/Users/gc3045/git/laml2-experiments/real_data/baseMemoir/inputs/baseMemoir_test_data.csv", index=False)